<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_abc/blob/main/PIFWIA_ABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ABC - Artificial Bee Colony ou Colônia Artificial de Abelhas é uma metaheurística proposta pelo pesquisador Derviş Karaboğa em 2005 para resolver problemas de otimização inspirado no comportamento do forrageamento inteligente das abelhas, buscando melhorar a solução candidata em relação a um objetivo.

Karaboga, D. An Idea Based on Honey Bee Swarm for Numerical Optimization Artificial Intelligence Research at the University of Bologna. http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.714.4934, 2005.

In [ ]:
#instala biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importa bibliotecas
import random
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from numpy.random import choice
from sklearn import metrics
import Orange

In [ ]:
#define os hiperparâmetros
DIMENSOES = 2 #determina a quantidade de dimensões do problema
ITERACOES = 200 #quantiddade máxima de ciclos defindo quantas explorações serão feitas (episódios)
ENXAME = 40 #tamanho da população correspondente ao número de abelhas na colônia
FONTES_ALIMENTACAO = int(ENXAME/2) #(nf) número de fontes de alimento que a colônia irá explorar (i.e., qtde de abelhas exploradoras) que é a metade da qtde de abelhas trabalhadoras
E1 = -15 #extremo esquerdo eixo x
E2 = 15 #extremo direito eixo x
E3 = -100 #extremo inferior eixo y
E4 = 100 #extremo superior eixo y
LIMITES = [E3,E4] #(bound) determina os valores maximos e minimos do espaço de busca (área de busca de fontes de alimentação)
CICLOS = 10 #(limite) qtde de ciclos sem melhora na exploração de uma fonte de alimento utilizado para abandonar essa exploração e buscar outras fontes aleatoriamente
FCUSTO = 'rosenbrock' #(fitness) função custo que pode ser uma esfera ou parábola (rosenbrock) ou outra função custo que defina a densidade de alimentação das abelhas na área de busca
ABELHAS = [] #(swarm) array da criação das abelhas

In [ ]:
#importa dados
from google.colab import files  
files.upload()

In [ ]:
#instancia objeto de dados com base no caminho gerado na importação do arquivo
dados = Orange.data.Table("/content/dados.csv")

In [ ]:
#explora os metadados e dados da arquivo importado
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)
dados.domain.attributes #explora os domínios dos atributos (campos da base de dados)
print("Qtde de Registros:", len(dados)) #explora os dados (quantidade de registros da base de dados)
i = 0 #exibe os primeiros registros para análise dos dados importados
for d in dados[:20]:
  i += 1
  print(i, d)

In [ ]:
#cria arrays das dimensões do problema a ser otimizado
periodo = []
complexidade = [] #1-muito baixa complexidade;2-baixa complexidade;3-média complexidade;4-alta complexidade;e,5-muito alta complexidade
pagina = []
prazo = []
revisao = []
entrega = []
i = 0
for d in dados[:ENXAME]:
  periodo.append(d[1])
  complexidade.append(d[2])
  pagina.append(d[3])
  prazo.append(d[4])
  revisao.append(d[5])
  entrega.append(d[6])
  print("id:",i,"período:",periodo[i],"complexidade:",complexidade[i],"página:",pagina[i],"prazo:",prazo[i],"revisões:",revisao[i],"entrega:",entrega[i])
  i += 1

In [ ]:
#abelha - unidade base da otimização, posicionada numa determinada posição no espaço de busca do problema, representando uma solução em potencial para o problema
class Exploradora: #(exploration) responsáveis por sair da colméia em busca de uma fonte de alimento para que de posse da informação da qualidade da fonte (fitness) retornam para repassar a informação
  def __init__(self):
        self.posicao = None
        self.trabalhadoras = []
        self.ciclos_sem_melhora = 0
class Trabalhadora: #(exploitation) escolhem uma fonte de alimento para explorar de acordo com seu fitness, sendo as responsáveis por explorar a vizinhança de uma fonte de alimento
  def __init__(self):
        self.posicao = None

In [ ]:
#função custo ou objetivo ou aptidão ou otimização ou fitness - usada para buscar o melhor ponto dentro de um espaço de buscao (melhor global) sem ficar preso em um melhor local
def fitness(problema, posicoes, alfa=0, beta=0):
  total = 0.0
  if problema == 'rosenbrock':
    for i in range(DIMENSOES-1):
      total += 100*(posicoes[i+1] - posicoes[i]**2)**2 + (1-posicoes[i])**2
  elif problema == 'esfera':
    for i in range(DIMENSOES):
      total += posicoes[i]**2
  elif problema == 'custo':
    for i in range(DIMENSOES-1):
      total += 1 / abs(sum([coord ** 2 for coord in posicoes]))
  elif problema == 'rota':
    denominador = sum([(caminho.feromonio)**alfa * (1 / caminho.comprimento)**beta for caminho in posicoes])
    distribuicao_probabilidades = None
    if denominador == 0:
      distribuicao_probabilidades = [1 / len(posicoes)  for _ in posicoes]
    else:
      distribuicao_probabilidades = [((caminho.feromonio)**alfa * (1 / caminho.comprimento)**beta) / denominador for caminho in posicoes]
    total = choice(posicoes, 1, p=distribuicao_probabilidades)[0]
  else:
    print('Problema não encontrado!')
  return total

In [ ]:
#inicializa população (enxame)
exploradoras = []
trabalhadoras = []
for i in range(FONTES_ALIMENTACAO):
  cpx = complexidade[i]
  pag = pagina[i]
  exploradora = Exploradora()
  exploradora.posicao = [cpx,pag] #[random.uniform(E1,E2), random.uniform(E1,E2)]
  exploradora.melhor_posicao = exploradora.posicao.copy()
  exploradoras.append(exploradora)
  trabalhadora = Trabalhadora()
  trabalhadoras.append(trabalhadora)
ABELHAS = exploradoras + trabalhadoras
#leitura das posições
for i in range(FONTES_ALIMENTACAO):
  print("i:",i)
  print(list(ABELHAS[i].posicao))
  print(list(ABELHAS[i].melhor_posicao))

In [ ]:
#posiciona as abelhas exploradoras no espaço de busca
print("Plano Cartesiano")
plt.axis([E1,E2,E3,E4])
plt.plot(0,0, marker='*', markersize=15, color='b')
for i in range(FONTES_ALIMENTACAO):
    abelha = exploradoras[i]
    #variáveis das DIMENSÕES do problema
    d1,d2 = zip(abelha.posicao)
    plt.plot(d1,d2, marker='o')
plt.show()

In [ ]:
#calcula escolha das fontes de alimento
def alocar_abelhas(abelhas):
  #faz a soma dos fitness das abelhas exploradoras
  soma_fitness = 0
  for i in range(FONTES_ALIMENTACAO):
    soma_fitness += fitness(FCUSTO, exploradoras[i].posicao)
  #cria a distribuição de probabilidades de acordo com o fitness calculado
  distribuicao_probabilidade = []
  for i in range(FONTES_ALIMENTACAO):
    probabilidade_alocacao = fitness(FCUSTO, exploradoras[i].posicao) / soma_fitness
    distribuicao_probabilidade.append(probabilidade_alocacao)
  #posiciona as oportunitas de acordo com a atratividade das exploradoras, "transformando-as" agora em trabalhadoras
  for abelha in abelhas:
    exploradora = choice(exploradoras, 1, p=distribuicao_probabilidade)[0]
    abelha.posicao = exploradora.posicao.copy()
    exploradora.trabalhadoras.append(abelha)

In [ ]:
#calcula movimento da abelha exploradora
def movimenta_abelha(abelha):
  colonia_candidata = ABELHAS.copy()
  colonia_candidata.remove(abelha)
  abelha_candidata = random.choice(colonia_candidata)
  x = abelha.posicao[0] + random.uniform(-1, 1) * (abelha.posicao[0] - abelha_candidata.posicao[0])
  y = abelha.posicao[1] + random.uniform(-1, 1) * (abelha.posicao[1] - abelha_candidata.posicao[1])
  if x < -10:
    x = -10
  elif x > 10:
    x = 10
  if y < -10:
    y = -10
  elif y > 10:
    y = 10
  if fitness(FCUSTO, [x, y]) > fitness(FCUSTO, abelha.posicao):
    abelha.posicao = [x, y]

In [ ]:
#otimiza as posições
melhor_posicao = None
ciclo = 0
#alocando as abelhas oportunistas como trabalhadoras em suas fontes
alocar_abelhas(trabalhadoras)
lista_melhores_valores = []
while ciclo < ITERACOES:
  print("Iteração: {:.0f}".format(ciclo+1))
  ciclo += 1
  for exploradora in exploradoras:
    #movimentando as trabalhadoras de cada fonte de comida
    for trabalhadora in exploradora.trabalhadoras:
      movimenta_abelha(trabalhadora)
    #obtem a melhor posição da iteração
    melhor_posicao_iter = None
    if len(exploradora.trabalhadoras) == 0:
      melhor_posicao_iter = exploradora.posicao
    for trabalhadora in exploradora.trabalhadoras:
      if melhor_posicao_iter is None or fitness(FCUSTO, trabalhadora.posicao) > fitness(FCUSTO, melhor_posicao_iter):
        melhor_posicao_iter = trabalhadora.posicao.copy()
    #atualiza posição atual da abelha exploradora
    if melhor_posicao_iter is not None and fitness(FCUSTO, melhor_posicao_iter) > fitness(FCUSTO, exploradora.posicao):
      exploradora.posicao = melhor_posicao_iter.copy()
      exploradora.ciclos_sem_melhora = 0
    else:
      exploradora.ciclos_sem_melhora += 1
    #desfaz a exploração da fonte de comida se necessário
    if exploradora.ciclos_sem_melhora >= CICLOS:
      exploradora.posicao = [random.uniform(-10, 10), random.uniform(-10, 10)]
      exploradora.melhor_posicao = exploradora.posicao.copy()
      exploradora.ciclos_sem_melhora = 0
      trabalhadoras_desalocadas = exploradora.trabalhadoras
      exploradora.trabalhadoras = []
      alocar_abelhas(trabalhadoras_desalocadas)
    #atuaiza melhor posicao global
    if melhor_posicao is None or fitness(FCUSTO, melhor_posicao_iter) > fitness(FCUSTO, melhor_posicao):
      melhor_posicao = melhor_posicao_iter.copy()
    lista_melhores_valores.append(melhor_posicao)
  plt.axis([E1,E2,E3,E4])
  plt.plot(0,0, marker='*', markersize=10, color='b')
  #exibe o enxame - mostrando no gráfico as posições atuais de cada abelha exploradora
  for i in range(FONTES_ALIMENTACAO):
    exploradora = exploradoras[i]
    d1,d2 = zip(exploradora.posicao)
    plt.plot(d1,d2, marker='o')
  #plt.plot(melhor_posicao[0], marker='x', markersize=15, color='r') #ponto gbest
  plt.show()
  print("Melhor Ponto: ", melhor_posicao)
  print("")

In [ ]:
#exibe das posições e otimizações calculadas
x = []
y = []
for i in range(len(lista_melhores_valores)):
  x.append(i)
  y.append(lista_melhores_valores[i])
plt.plot(x, y)
plt.title("Curva de Convergência do ABC")
plt.xlabel("Tempo")
plt.ylabel("Ciclos")
plt.tight_layout()

In [ ]:
#leitura das posições e otimização
posicao_inicial = []
posicao_otimizada = []
for i in range(ENXAME):
  posicao_inicial.append(ABELHAS[i].posicao)
  posicao_otimizada.append(lista_melhores_valores[i])
  print("i:",i,"posição:",posicao_inicial[i],"otimização:",posicao_otimizada[i])
#y = []
#ỹ = []
#for i in range(ENXAME):
#  y.append(int(posicao_inicial[i][0]))
#  ỹ.append(int(posicao_otimizada[i][0]))
#print('A:', metrics.accuracy_score(y,ỹ))
#print('P:', metrics.precision_score(y,ỹ,average='macro'))
#print('R:', metrics.recall_score(y,ỹ,average='macro'))
#print('F:', metrics.f1_score(y,ỹ,average='macro'))